# Scrapping Dow Jones Index

In [14]:
# using Selenium since the point on the graph found on https://fred.stlouisfed.org/series/DJIA appear when you hover over the line on graph
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from IPython.display import clear_output

import os
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

driver = webdriver.Chrome(chromedriver)
driver.get("https://fred.stlouisfed.org/series/DJIA")

In [15]:
# this allows us to select and input our date range in the graph settings
driver.find_element_by_id('input-cosd').click()
time.sleep(2)
driver.find_element_by_xpath("//span[text()='2016']").click()
time.sleep(2)
driver.find_element_by_xpath("//span[text()='Dec']").click()
time.sleep(2)
driver.find_element_by_xpath("//td[text()='26']").click()

In [16]:
# selecting graph space
svg=driver.find_element_by_tag_name("svg")

In [17]:
rect = svg.find_element_by_class_name('highcharts-plot-background')

In [18]:
width = int(rect.get_attribute('width'))

In [19]:
# initializing our list to collect our data
djia_list = []

In [20]:
# initializing scrapping through graph
rect.click()
action = webdriver.common.action_chains.ActionChains(driver)
for i, x in enumerate(range(width)):
#     clear_output(wait=True)
#     print('%d/%d' % (i, width))
    action.move_to_element_with_offset(rect, x, 20)
    action.click()
    action.perform()
    action.reset_actions()
    # collecting the data
    djia_list.append(driver.find_elements_by_class_name('highcharts-tooltip')[-1].find_element_by_tag_name('span').text.split(': '))

In [8]:
# putting the data in dija_list into panda data frame
import pandas as pd
df = pd.DataFrame(djia_list, columns=['Date', 'DayIndex'])
df.head()

,Date,DayIndex
0,"Tue, Dec 27, 2016","19,945.04"
1,"Tue, Dec 27, 2016","19,945.04"
2,"Tue, Dec 27, 2016","19,945.04"
3,"Tue, Dec 27, 2016","19,945.04"
4,"Wed, Dec 28, 2016","19,833.68"


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1049 entries, 0 to 1048
Data columns (total 2 columns):
Date        1049 non-null object
DayIndex    1049 non-null object
dtypes: object(2)
memory usage: 16.5+ KB


In [10]:
# cleaning data. Switiching date strings to datetime objects 
df['Date'] = pd.to_datetime(df['Date'])

# switching the index values from string to float and removing commas
df['DayIndex'] = df['DayIndex'].apply(lambda x: float(x.replace(',', '')))
#df.set_index('Date', inplace=True)

In [11]:
# dropping duplicates since there are many. for loop checked at every pixel until end of graph to ensure no points were missed.
df.drop_duplicates(inplace = True)
df.sort_values(by= 'Date').head()


,Date,DayIndex
0,2016-12-27,19945.04
4,2016-12-28,19833.68
6,2016-12-29,19819.78
8,2016-12-30,19762.60
14,2017-01-03,19881.76


In [12]:
# plot to check 
df.set_index('Date', inplace=True)
df.plot()

In [13]:
# If I just wanted to click the csv button and download my data 

#driver.find_elements_by_class_name('highcharts-tooltip')
#button = driver.find_element_by_id('download-button')
#button.click()
#data = driver.find_element_by_id('download-data-csv')
#data.click()